# Trait
---

- trait abstracts over classes, just as classes abstracts over objects.
- it allows you to write down the template that classes extending it shares.
- it can have `abstract` as well as `concrete` members 
    - abstract member: only identifier and types are declared, implementation is not given
    - concrete member: members with values (or implementation)

### in absence of trait construct
---

Let's define two classes which are related in some way, and almost shares same properties

In [1]:
case class Developer(name: String, language: String)

case class Designer(name: String, language: String, framework: String)

defined class Developer
defined class Designer

As we can see, both classes share `name` and `language` but Designer class only has `framework` property.

1. `name` and `language` doesn't seem repetitive in this case, both are values and shared between two classes only. But think about some method which is ten lines long and needs to copy across five classes, that _is_ verbose.
2. Even though we know that both classes are related, Scala won't allow us to write a method which expectes one of the classes. The only way to implement such a method would be to have a parameter of type `Any` (or `AnyRef`), but it'll have its own problem. Such as, Scala will allow us to pass a type of value to it.

In [2]:
def printLanguage(value: Any): Unit = ???

// printLanguage(Developer("Dennis", "C")) => "C"
// printLanguage(42.0)                     => THIS DOESN'T MAKE SENSE YET SCALA WILL COMPILE THE SOURCE!

defined function printLanguage

### trait construct to the rescue
---

The two problems are (1) repetition of code (2) no precise supertype to write operation on them. The `trait` construct helps us to resolve them.

As mentioned above, trait allows us to write down a common template for classes (solves the first problem) and by extending our classes with a trait, we are suggesting to Scala that both are related, that we give them common supertype (solves the second problem).

- Syntax: `<class declaration> extends <trait>`

In [3]:
trait Programmer {
    // prefer `def` over `val` in trait declaration
    def name: String
    def language: String
    
    def intro: String = name + " uses " + language + " programming language."
}

case class Developer(name: String, language: String) extends Programmer

case class Designer(name: String, language: String, framework: String) extends Programmer

defined trait Programmer
defined class Developer
defined class Designer

- `Programmer` is supertype of `Developer` and `Designer`, and they are its subtype.
- all classes that are extending the `Programmer` trait will have `name`, `language` and `intro` properties.
- `name` and `language` are abstract members whereas `intro` is concrete.
- subtype will have all the members of its supertype but can hold more, like the `framework` property of `Designer` class.
- subtype can `override` the implementation of supertype's concrete member.

In [4]:
case class Designer(name: String, language: String, framework: String) extends Programmer {
    override def intro: String = name + " works on " + framework + " framework and uses " + language + " programming language."
}

val developer = Developer("Dennis", "C")
val designer = Designer("Evan", "TypeScript", "Vue.js")

defined class Designer
developer: Developer = Developer(name = "Dennis", language = "C")
designer: Designer = Designer(
  name = "Evan",
  language = "TypeScript",
  framework = "Vue.js"
)

In [5]:
println(developer.intro)
println(designer.intro)

Dennis uses C programming language.
Evan works on Vue.js framework and uses TypeScript programming language.


It was the example of overwriting the supertype's method. The `Designer` class's `intro` method prints a different message than `Developer` class.

Let's see how common supertype helps us to implement methods on those two classes, using pattern matching.

In [6]:
def printLanguage(p: Programmer): Unit = p match {
    case Developer(_, language) => println(language)
    case Designer(_, language, _) => println(language)
}

defined function printLanguage

In [7]:
printLanguage(developer)
printLanguage(designer)
// printLanguage(42.0) will throw error: [found: Double, required: Programmer]

C
TypeScript


#### trait vs class
---

- trait can have abstract members, class can't<sup>1</sup>.
- we can't create objects using trait

### sealed and final modifiers
---

These modifiers provide more information to Scala about a trait and classes that are extending it
1. If a trait is `sealed`, Scala will know that all the classes that are extending it are in the same file. It won't compile the project if class is declared in another file and extends the trait.
2. If a class is declared as `final`, then no one can extend it further.
3. These modifiers give us tight control over who can extend our data structure, and provides Scala enough information to flag some errors. For example, in pattern matching.

In [8]:
// same as above but with `sealed` and `final` modifiers

sealed trait Programmer {
    def name: String
    def language: String
    
    def intro: String = name + " uses " + language + " programming language."
}

final case class Developer(name: String, language: String) extends Programmer

final case class Designer(name: String, language: String, framework: String) extends Programmer

defined trait Programmer
defined class Developer
defined class Designer

Let's see how Scala helps us in pattern matching,

```
def printLanguage(p: Programmer): Unit = p match {
    case Developer(_, language) => println(language)
    // we forget to add Designer case
}

// warning: match may not be exhaustive.
// It would fail on the following input: Designer(_, _, _)
```

Because we declared the trait as `sealed` and Scala knows that two classes are extending it, it throws a warning on the missed match case. 

---

[1] abstract class can.

---

### Exercise
---

1) Define a trait called `Shape` and give it three methods.
    - `sides` returns the number of sides
    - `perimeter` returns the total length of the sides
    - `area` returns the area
    
    Implement `Shape` with three classes: `Circle`, `Rectangle` and `Square`, and provide implementations of each of the three methods.

2) Write a method in the `Shape` object to return the `area` of any provided `Shape` using pattern matching.

3) What's the use of `sealed` and `final` modifiers?

4) Use `sealed` and `final` modifiers in the first exercise (if not already), and try to make a fourth class implementing the `Shape` in a separate file. What errors Scala raises? 

5) Why is it beneficial to define `def` in trait, rather than `val`?